In [1]:
import tensorflow as tf
import keras 
from keras.applications import VGG16
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator

Using TensorFlow backend.


In [2]:
conv_base = VGG16(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [ ]:
# conv_base.summary()

In [3]:
import pickle
X_train = pickle.load(open(f'pickle_files/X298.pkl', 'rb'))
X_train = X_train.permute(0,2,3,1).numpy()


In [4]:
X_train.shape, type(X_train)


((298, 300, 300, 3), numpy.ndarray)

In [5]:
nos = X_train.shape[0]

In [6]:
y_train = pickle.load(open(f'pickle_files/y298.pkl', 'rb'))
y_train = y_train.numpy()
y_train.shape, type(y_train)

((298,), numpy.ndarray)

In [7]:
datagen = ImageDataGenerator(rescale=1./255)

In [8]:
bs = 64
numit = NumpyArrayIterator(
    X_train, y_train, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

In [25]:
samples = np.zeros(shape=(nos,9,9,512))
i = 0
for a,b in numit:
    samples[i*bs:(i+1)*bs] = conv_base.predict(a)
    i+=1
    if (i*bs)>nos:
        break

In [27]:
train_samples = samples.reshape(nos,9*9*512)
train_samples.shape

(298, 41472)

In [28]:
from keras import models, layers, optimizers

In [29]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 2e-5)),
              loss = 'binary_crossentropy',
              metrics = ['acc'])
        


In [ ]:
history = model.fit(train_samples, train_labels, epochs = 30, batch_size = 20,
                    validation_data = (validation_features, validation_labels)
)